Reference: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9302620

## 1. Load trajectory data

In [1]:
import pandas as pd
import os
import numpy as np

data_path = "trajectory.csv"
traj = pd.read_csv(data_path)

coords = []
for i in range(len(traj)):
    if traj.Visibility[i] == 1:
        coords.append([traj.X[i], traj.Y[i]])
    else:
        coords.append(None)


## 2. Visualize shuttlecock

In [2]:
import cv2


def plot_shuttlecock(
    coords,
    video_path="input.mp4",
    output_path="output.mp4",
):
    cap = cv2.VideoCapture(video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (1280, 720))

    for i in range(len(coords)):
        ret, frame = cap.read()
        if not ret:
            break

        if coords[i] is not None:
            cv2.circle(
                frame, (int(coords[i][0]), int(coords[i][1])), 5, (0, 0, 255), -1
            )

        out.write(frame)

    cap.release()
    out.release()


plot_shuttlecock(coords, output_path="tracknet_prediction.mp4")


## 3. Denoise

In [3]:
import math

before_coords = coords.copy()

# Mark shifted coordinates
shifted_coords = []
for i in range(1, len(coords) - 1):
    if coords[i - 1] is None or coords[i] is None or coords[i + 1] is None:
        continue
    if (
        math.dist(coords[i - 1], coords[i]) > 100
        and math.dist(coords[i], coords[i + 1]) > 100
    ):
        shifted_coords.append(i)

# Remove shifted coordinates
for i in shifted_coords:
    coords[i] = None

plot_shuttlecock(coords, output_path="denoised_trajectory.mp4")


In [4]:
print("Number of points before denoising:", len([coord for coord in before_coords if coord is not None]))
print("Number of points after denoising:", len([coord for coord in coords if coord is not None]))

Number of points before denoising: 1149
Number of points after denoising: 1146


## 4. Curve fitting

In [5]:
before_coords = coords.copy()

def shortest_distance_to_curve(x, y, curve_coef):
    a = curve_coef[2]
    b = curve_coef[1]
    c = curve_coef[0]

    # Reference: https://mathworld.wolfram.com/Point-QuadraticDistance.html
    x_candidates = np.roots(
        [2 * (a**2), 3 * a * b, b**2 + 2 * a * c - 2 * a * y + 1, c * b - y * b - x]
    )
    curve_func = np.poly1d(curve_coef[::-1])
    return min(
        [
            math.dist([x_candidate, curve_func(x_candidate)], [x, y])
            for x_candidate in x_candidates
        ]
    )


front_check_dist = [None] * len(coords)
back_check_dist = [None] * len(coords)

for i in range(len(coords) - 7):
    curve_coords = np.array([coord for coord in coords[i : i + 7] if coord is not None])

    if len(curve_coords) < 3:
        continue

    x = curve_coords[:, 0]
    y = curve_coords[:, 1]
    curve_coef = np.polynomial.polynomial.polyfit(x, y, 2)

    if i != 0 and coords[i - 1] is not None:
        front_check_dist[i - 1] = shortest_distance_to_curve(
            coords[i - 1][0], coords[i - 1][1], curve_coef
        )

    if i != len(coords) - 7 and coords[i + 7] is not None:
        back_check_dist[i + 7] = shortest_distance_to_curve(
            coords[i + 7][0], coords[i + 7][1], curve_coef
        )

for i, (f_dist, b_dist) in enumerate(zip(front_check_dist, back_check_dist)):
    if f_dist is not None and b_dist is not None and f_dist > 100 and b_dist > 100:
        coords[i] = None


x = np.array([coord[0] if coord is not None else np.nan for coord in coords])
y = np.array([coord[1] if coord is not None else np.nan for coord in coords])

for i, (f_dist, b_dist) in enumerate(zip(front_check_dist, back_check_dist)):
    if f_dist is not None and b_dist is not None and f_dist < 5 and b_dist < 5:
        # Fill missing points in the front check window
        missing = np.isnan(x[i - 7 : i + 1])
        x[i - 7 : i + 1][missing] = np.interp(
            np.nonzero(missing)[0], np.nonzero(~missing)[0], x[i - 7 : i + 1][~missing], left=np.nan, right=np.nan
        )

        curve_coords = np.array([coord for coord in coords[i - 7 : i + 1] if coord is not None])
        curve_x = curve_coords[:, 0]
        curve_y = curve_coords[:, 1]
        curve_coef = np.polynomial.polynomial.polyfit(curve_x, curve_y, 2)

        y[i - 7 : i + 1][missing] = np.poly1d(curve_coef[::-1])(x[i - 7 : i + 1][missing])

        # Fill missing points in the back check window
        missing = np.isnan(x[i : i + 8])
        x[i : i + 8][missing] = np.interp(
            np.nonzero(missing)[0], np.nonzero(~missing)[0], x[i : i + 8][~missing], left=np.nan, right=np.nan
        )

        curve_coords = np.array([coord for coord in coords[i : i + 8] if coord is not None])
        curve_x = curve_coords[:, 0]
        curve_y = curve_coords[:, 1]
        curve_coef = np.polynomial.polynomial.polyfit(curve_x, curve_y, 2)

        y[i : i + 8][missing] = np.poly1d(curve_coef[::-1])(x[i : i + 8][missing])  

coords = [*zip(x, y)]
coords = [None if np.isnan(coord[0]) else coord for coord in coords]

plot_shuttlecock(coords, output_path="curve_fitting.mp4")


/home/jerrylee/.miniconda3/envs/aicup/lib/python3.9/site-packages/numpy/polynomial/polynomial.py:1362: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)
/tmp/ipykernel_1154751/3242081086.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  math.dist([x_candidate, curve_func(x_candidate)], [x, y])


In [6]:
print("Number of points before curve fitting: ", len([coord for coord in before_coords if coord is not None]))
print("Number of points after curve fitting: ", len([coord for coord in coords if coord is not None]))

Number of points before curve fitting:  1146
Number of points after curve fitting:  1218


## 5. Interpolation

In [7]:
before_coords = coords.copy()

interp_begin = None

x = np.array([coord[0] if coord is not None else np.nan for coord in coords])
y = np.array([coord[1] if coord is not None else np.nan for coord in coords])

for i in range(3, len(coords) - 3):
    if coords[i] is not None:
        continue

    if interp_begin is None:
        if not None in coords[i - 3 : i]:
            interp_begin = i - 3
    else:
        if i - interp_begin > 8:
            interp_begin = None

    if interp_begin is not None and not None in coords[i + 1 : i + 4]:
        interp_end = i + 4
        missing = np.isnan(x[interp_begin:interp_end])
        x[interp_begin:interp_end][missing] = np.interp(
            np.nonzero(missing)[0],
            np.nonzero(~missing)[0],
            x[interp_begin:interp_end][~missing],
        )

        curve_coords = np.array(
            [coord for coord in coords[interp_begin:interp_end] if coord is not None]
        )
        curve_x = curve_coords[:, 0]
        curve_y = curve_coords[:, 1]
        curve_coef = np.polynomial.polynomial.polyfit(curve_x, curve_y, 2)

        y[interp_begin:interp_end][missing] = np.poly1d(curve_coef[::-1])(
            x[interp_begin:interp_end][missing]
        )

        interp_begin = None

coords = [*zip(x, y)]
coords = [None if np.isnan(coord[0]) else coord for coord in coords]

plot_shuttlecock(coords, output_path="interpolation.mp4")


In [8]:
print("Number of points before interpolation: ", len([coord for coord in before_coords if coord is not None]))
print("Number of points after interpolation: ", len([coord for coord in coords if coord is not None]))

Number of points before interpolation:  1218
Number of points after interpolation:  1236
